In [1]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'cashup_backend.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

import django
django.setup()

In [2]:
from data.models import MinuteData, HourData, UpFlow, DownFlow

In [3]:
import requests
from datetime import timedelta, datetime

In [4]:
up_dn_list = []
last_data = []
dn_list = []
up_list = []
new_dn_list = []
new_up_list = []
other_up_dn_list = []

continue_up_down = "D0"
prev_up_down = "S"
work_1_0_1 = "S"
work_1_5_1 = "S"
work_1_0_2 = "S"
work_1_5_2 = "S"
last_time_1_0_1 = datetime.now()
last_time_1_5_1 = datetime.now()
last_time_1_0_2 = datetime.now()
last_time_1_5_2 = datetime.now()

last_dn, last_up = "", ""

max_price, min_price = 0, 0
work_up_high, work_dn_low = 1000000, 0
volume = 0
prev_low = 0
prev_high = 100000
start = 0
last_up = 0
last_dn = 0
volume_up_dn = ""
hour_up_down = ""
prev_signal_down_price = 0
prev_signal_up_price = 0

base_price = 0
flow_base_price = 0
prev_signal = ""
flow_list = []
flow_element_list = []
prev_down_flow = 0
prev_up_flow = 0
down_flow = ['DN', 'DN', 'UP', 'UP']
up_flow = ['UP', 'UP', 'DN', 'DN']
down_flow_list = []
up_flow_list = []
last_down_flow_time = ""
last_up_flow_time = ""
down_flow_element_list = []
down_flow_confirm_list = []
up_flow_element_list = []
up_flow_confirm_list = []

In [ ]:
while True:
    URL = "https://www.bitmex.com/api/v1/trade/bucketed?symbol=XBT&binSize=5m&partial=false&start={}&count=1000&reverse=false".format(start)
    req = requests.get(URL).json()
    for data in req:
        volume_up_dn = ""
        signal = ""
        signal_price = 0
        hour_up_down = ""
        now_work_1_0_1 = ""
        now_work_1_5_1 = ""
        now_work_1_0_2 = ""
        now_work_1_5_2 = ""
        datetime = datetime.strptime(data['timestamp'].replace("T", " ")[0:19], "%Y-%m-%d %H:%M:%S") + timedelta(hours=9) - timedelta(minutes=5)
        open_price = data['open'] if data['open'] != None else 0
        high_price = data['high'] if data['high'] != None else 0
        low_price = data['low'] if data['low'] != None else 0
        close_price = data['close'] if data['close'] != None else 0
        avg_price = (high_price + low_price) / 2
        
        if volume != 0:
            volume_rate = data['volume'] / volume
        else:
            volume_rate = 0
            
        volume = data['volume']

        flag = False

        if open_price > close_price:
            UD = "D"

        elif open_price < close_price:
            UD = "U"

        else:
            UD = "S"

        if len(last_data) > 3:
            last_data.pop(0)
            volume_bigger_count = 0
            volume_smaller_count = 0

            for i in last_data:
                if i[2] < volume:
                    volume_bigger_count += 1

                if i[2] > volume:
                    volume_smaller_count += 1

            if volume_bigger_count == 3:
                volume_up_dn = "UP"

            if volume_smaller_count == 3:
                volume_up_dn = "DN"

        last_data.append([low_price, high_price, volume])
        
        if continue_up_down[0] == "U":
            if UD == "U":
                continue_up_down = "U" + str(int(continue_up_down[1:]) + 1)
            elif UD == "D":
                continue_up_down = "D1"
        else:
            if UD == "U":
                continue_up_down = "U1"
            elif UD == "D":
                continue_up_down = "D" + str(int(continue_up_down[1:]) + 1)
        
        if len(up_dn_list) > 3:
            up_dn_list.pop(0)
        up_dn_list.append(UD)
        
        if up_dn_list.count("U") > 2:
            if high_price >= last_up:
                hour_up_down = "UP(U)"
            else:
                hour_up_down = "UP(D)"
            last_up = high_price
        
        if up_dn_list.count("D") > 2:
            if low_price >= last_dn:
                hour_up_down = "DN(U)"
            else:
                hour_up_down = "DN(D)"
            last_dn = low_price
        
        if UD == "U":
            if other_up_dn_list.count("D") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                is_U = False
                for idx, element in enumerate(other_up_dn_list):
                    if element == "D":
                        count += 1
                        max_price_list.append(dn_list[idx])
                    else:
                        is_U = True
                    if is_U and count >= 3:
                        if element == "U":
                            break
                        else:
                            min_price_list.append(up_list[idx])
                            break
                    min_price_list.append(up_list[idx])
                if prev_up_down == "D" and max(max_price_list) == max_price_list[len(max_price_list) - 1]:
                    if prev_signal_down_price != 0:
                        if prev_signal_down_price < min(min_price_list):
                            signal = "fD(U)"
                        elif prev_signal_down_price == min(min_price_list):
                            signal = "fD(0)"
                        else:
                            signal = "fD(D)"
                    prev_signal_down_price = min(min_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()

        if UD == "D":
            if other_up_dn_list.count("U") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                is_D = False
                for idx, element in enumerate(other_up_dn_list):
                    if element == "U":
                        count += 1
                        min_price_list.append(up_list[idx])
                    else:
                        is_D = True

                    if is_D and count >= 3:
                        if element == "D":
                            break
                        else:
                            max_price_list.append(dn_list[idx])
                            break
                    max_price_list.append(dn_list[idx])

                if prev_up_down == "U" and min(min_price_list) == min_price_list[len(min_price_list) - 1]:
                    print(datetime)
                    print(other_up_dn_list)
                    print(max_price_list, min_price_list)
                    if prev_signal_down_price != 0:
                        if prev_signal_up_price < max(max_price_list):
                            signal = "fU(U)"
                        elif prev_signal_up_price == max(max_price_list):
                            signal = "fU(0)"
                        else:
                            signal = "fU(D)"
                    prev_signal_up_price = max(max_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                    
        if len(dn_list) > 5:
            dn_list.pop(0)
        dn_list.append(high_price)
            
        if len(up_list) > 5:
            up_list.pop(0)
        up_list.append(low_price)
        
        if len(other_up_dn_list) > 5:
            other_up_dn_list.pop(0)
        other_up_dn_list.append(UD)
        
        if volume_up_dn == "UP":
            if (max_price < high_price) and (volume_rate > 1):
                if work_1_0_1[0] == "U":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0_1 = True
                    else:
                        work_1_0_1 = "U" + str(int(work_1_0_1[1:]) + 1)
                else:
                    work_1_0_1 = "U1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1):
                if work_1_0_1[0] == "D":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0 = True
                    else:
                        work_1_0_1 = "D" + str(int(work_1_0_1[1:]) + 1)
                
                else:
                    work_1_0_1 = "D1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
                
            if (max_price < high_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "U":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        continue_flag_1_5_1 = True
                    else:
                        work_1_5_1 = "U" + str(int(work_1_5_1[1:]) + 1)
                else:
                    work_1_5_1 = "U1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_1 = "D" + str(int(work_1_5_1[1:]) + 1)
                
                else:
                    work_1_5_1 = "D1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime

            if (max_price < high_price and min_price < low_price) and (volume_rate > 1):
                if work_1_0_2[0] == "U":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "U" + str(int(work_1_0_2[1:]) + 1)
                else:
                    work_1_0_2 = "U1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1):
                if work_1_0_2[0] == "D":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "D" + str(int(work_1_0_2[1:]) + 1)
                
                else:
                    work_1_0_2 = "D1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime     
                
                
            if (max_price < high_price and min_price < low_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "U":
                    if datetime == last_time_1_5_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "U" + str(int(work_1_5_2[1:]) + 1)
                else:
                    work_1_5_2 = "U1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "D" + str(int(work_1_5_2[1:]) + 1)
                
                else:
                    work_1_5_2 = "D1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime

            max_price, min_price = high_price, low_price
            
        if now_work_1_0_1 != "":
            now_work_1_0_1 = "w" + now_work_1_0_1
        
        if now_work_1_5_1 != "":
            now_work_1_5_1 = "w" + now_work_1_5_1
            
        if now_work_1_0_2 != "":
            now_work_1_0_2 = "w" + now_work_1_0_2
            
        if now_work_1_5_2 != "":
            now_work_1_5_2 = "w" + now_work_1_5_2
        
        flow = ""
        if flow_base_price != 0:
            if prev_signal == "fU(D)":
                if low_price > flow_base_price:
                    flow = "UP"
                else:
                    flow = "DN"
            if prev_signal == "fD(U)":
                if high_price > flow_base_price:
                    flow = "UP"
                else:
                    flow = "DN"

        if signal == "fU(D)":
            flow_base_price = high_price
            prev_signal = signal

        if signal == "fD(U)":
            flow_base_price = low_price
            prev_signal = signal

        if flow != "":
            if len(flow_list) > 3:
                flow_list.pop(0)
                flow_element_list.pop(0)
            flow_list.append(flow)
            flow_element_list.append({
                'datetime': datetime,
                'min_price': low_price,
                'max_price': high_price
            })
            
        flow_element = None

        if flow_list == down_flow:
            if prev_down_flow != 0:
                flow_element = DownFlow()
                flow_element.datetime = flow_element_list[1]['datetime']
                flow_element.base_price = flow_element_list[1]['max_price']
                if prev_down_flow < flow_element_list[1]['max_price']:
                    flow_element.down_flow = "UP"
                else:
                    flow_element.down_flow = "DN"
                if len(down_flow_list) > 4:
                    down_flow_list.pop(0)
                down_flow_list.append(flow_element.down_flow)

                if len(down_flow_list) > 4:
                    if down_flow_list.count("UP") > 3:
                        flow_element.down_flow_confirm = "UP"

                    if down_flow_list.count("DN") > 3:
                        flow_element.down_flow_confirm = "DN"
                        
                    if down_flow_list.count("UP") == 5:
                        flow_element.down_flow_trade = "B#4"                      

            prev_down_flow = flow_element_list[1]['max_price']

        if flow_list == up_flow:
            if prev_up_flow != 0:
                flow_element = UpFlow()
                flow_element.datetime = flow_element_list[1]['datetime']
                flow_element.base_price = flow_element_list[1]['min_price']
                if prev_up_flow < flow_element_list[1]['min_price']:
                    flow_element.up_flow = "UP"
                else:
                    flow_element.up_flow = "DN"

                if len(up_flow_list) > 4:
                    up_flow_list.pop(0)
                up_flow_list.append(flow_element.up_flow)

                if len(up_flow_list) > 4:
                    if up_flow_list.count("UP") > 3:
                        flow_element.up_flow_confirm = "UP"

                    if up_flow_list.count("DN") > 3:
                        flow_element.up_flow_confirm = "DN"
                    
                    if up_flow_list.count("DN") == 5:
                        flow_element.up_flow_trade = "S#4"

            prev_up_flow = flow_element_list[1]['min_price']

        if len(flow_element_list) > 3:
            if flow_element:
                flag = False
                if type(flow_element) == UpFlow:
                    flag = True
                
                if not flag and flow_element.down_flow_confirm == "UP":
                    last_down_flow_time = flow_element_list[1]['datetime']

                if flag and flow_element.up_flow_confirm == "DN":
                    last_up_dlow_time = flow_element_list[1]['datetime']

                if not flag and flow_element.down_flow:
                    if last_down_flow_time != "":
                        if flow_element_list[1]['datetime'] - last_down_flow_time > timedelta(hours=2):
                            if HourData.objects.filter(datetime__range=[last_down_flow_time, flow_element_list[1]['datetime'] - timedelta(hours=1)], up_down='U').values('up_down').count() > 1:
                                flow_element.down_flow_trade = "S#1"
                                last_down_flow_time = ""

                    elif down_flow_list[-2:] == ['UP', 'UP']:
                        flow_element.down_flow_trade = "S#3"

                    if len(down_flow_element_list) > 1:
                        down_flow_element_list.pop(0)
                    down_flow_element_list.append(flow_element)
                    
                    
                    if len(down_flow_element_list) > 1:
                        if down_flow_element_list[0].down_flow_confirm is None and flow_element.down_flow_confirm == "DN":
                            flow_element.down_flow_trade = "B#2"

                if flag and flow_element.up_flow:
                    if last_up_flow_time != "":
                        if flow_element_list[1]['datetime'] - last_up_flow_time > timedelta(hours=2):
                            if HourData.objects.filter(datetime__range=[last_up_flow_time, flow_element_list[1]['datetime'] - timedelta(hours=1)], up_down='D').values('up_down').count() > 1:
                                flow_element.up_flow_trade = "B#1"
                                last_up_flow_time = ""

                    elif up_flow_list[-2:] == ['DN', 'DN']:
                        flow_element.up_flow_trade = "B#3"

                    if len(up_flow_element_list) > 1:
                        up_flow_element_list.pop(0)
                    up_flow_element_list.append(flow_element)

                    if len(up_flow_element_list) > 1:
                        if up_flow_element_list[0].up_flow_confirm is None and flow_element.up_flow_confirm == "UP":
                            flow_element.down_flow_trade = "S#2"
                            
                            
        if flow_element:
            flow_element.save()

        element = MinuteData()
        element.datetime = datetime
        element.open_price = open_price
        element.min_price = low_price
        element.max_price = high_price
        element.close_price = close_price
        element.avg_price = avg_price
        element.volume = volume
        element.flow = flow
        element.volume_rate = volume_rate
        element.volume_up_dn = volume_up_dn
        element.up_down = UD
        element.continue_up_down = continue_up_down
        element.hour_up_down = hour_up_down
        element.signal = signal
        element.signal_price = signal_price
        element.work_two_1_0 = now_work_1_0_1
        element.work_two_1_5 = now_work_1_5_1
        element.work_one_1_0 = now_work_1_0_2
        element.work_one_1_5 = now_work_1_5_2
        element.save()
        
        prev_up_down = UD
        
        
    start += 1000

2015-09-26 03:50:00
['U', 'U', 'U', 'U', 'S', 'S']
[236.11, 236.08, 236.01, 236, 235.85] [236.08, 236.01, 236, 236]
2015-09-26 06:55:00
['U', 'D', 'U', 'U', 'D', 'S']
[234.94, 234.9, 234.94, 234.83] [234.94, 234.88, 234.83]
2015-09-26 21:35:00
['U', 'U', 'D', 'U', 'S', 'U']
[234.87, 234.8, 234.34, 234.4] [234.87, 234.47, 234.4]
2015-09-28 12:10:00
['U', 'D', 'S', 'U', 'U', 'S']
[234.96, 234.91, 234.58, 234.58, 234.11] [234.6, 234.58, 234.06]
2015-09-28 12:50:00
['U', 'U', 'U', 'U', 'U', 'U']
[237.86, 238.23, 236.9, 236.5, 235.99, 234.9] [237.64, 237.53, 236.62, 236.05, 235.65, 234.85]
2015-09-28 16:30:00
['U', 'U', 'S', 'S', 'U', 'S']
[238.7, 238.2, 237.3, 237.3, 237.3] [238.39, 237.53, 237.3]
2015-09-28 17:30:00
['U', 'U', 'S', 'S', 'D', 'U']
[239.55, 239.3, 238.8, 238.8, 240.49, 239] [239.21, 239.28, 239]
2015-09-29 08:50:00
['U', 'D', 'U', 'D', 'D', 'U']
[240.59, 240.49, 240.57, 240.3, 240.5, 240.63] [240.37, 240.21, 240.08]
2015-09-29 09:45:00
['U', 'D', 'U', 'U', 'D', 'U']
[241.5,

2015-10-17 18:00:00
['U', 'U', 'U', 'U', 'U', 'S']
[270.74, 269.94, 269.83, 269.46, 268.48, 268.37] [270.14, 269.79, 269.56, 268.53, 268.2]
2015-10-17 18:20:00
['U', 'U', 'U', 'D', 'U', 'U']
[271.37, 271.21, 271.04] [271.19, 271.06, 270.99]
2015-10-17 19:35:00
['U', 'U', 'D', 'U', 'U', 'U']
[271.1, 270.83, 270.5, 270.57] [270.9, 270.83, 269.95]
2015-10-17 20:10:00
['U', 'U', 'U', 'D', 'S', 'D']
[271.26, 271.16, 270.74] [270.99, 271.16, 270.43]
2015-10-17 20:30:00
['U', 'U', 'S', 'D', 'U', 'U']
[271.81, 271.2, 271.05, 271.05, 271.26] [271.59, 271.2, 270.99]
2015-10-17 20:40:00
['U', 'D', 'U', 'U', 'S', 'D']
[272.09, 271.59, 271.81, 271.2] [271.56, 271.59, 271.2]
2015-10-17 21:40:00
['U', 'U', 'U', 'S', 'S', 'D']
[271.6, 270.58, 270.35, 270.2] [271.3, 270.58, 270.35]
2015-10-17 23:00:00
['U', 'D', 'U', 'U', 'U', 'S']
[275, 273.93, 273.56, 271.42] [273.51, 271.46, 270.9]
2015-10-17 23:10:00
['U', 'D', 'U', 'D', 'U', 'U']
[274.6, 274.5, 275, 273.93, 273.56] [274.11, 273.51, 271.46]
2015-10

2015-10-22 17:25:00
['U', 'U', 'U', 'S', 'S', 'S']
[272.19, 271.17, 270.62, 270.42] [271.52, 270.91, 270.62]
2015-10-22 17:35:00
['U', 'D', 'U', 'U', 'U', 'S']
[272.43, 272.15, 272.19, 271.17] [272.18, 271.52, 270.91]
2015-10-22 19:00:00
['U', 'U', 'S', 'U', 'D', 'S']
[274.99, 273.47, 272.45, 272.45] [273.55, 272.92, 272.27]
2015-10-22 19:15:00
['U', 'S', 'D', 'U', 'U', 'S']
[276.01, 274.74, 275.8, 274.99, 273.47] [274.91, 273.55, 272.92]
2015-10-22 19:25:00
['U', 'D', 'U', 'S', 'D', 'U']
[276, 275.15, 276.01, 274.74, 275.8, 274.99] [274.23, 274.91, 273.55]
2015-10-22 20:25:00
['U', 'U', 'U', 'U', 'S', 'U']
[279.8, 277.4, 276.56, 276.02, 275.63] [277.08, 275.98, 275.7, 275.41]
2015-10-22 23:05:00
['U', 'S', 'U', 'S', 'U', 'D']
[277.61, 276.89, 276.89, 276.36, 276.36] [277.12, 276.34, 276.22]
2015-10-22 23:35:00
['U', 'S', 'U', 'U', 'D', 'D']
[276.66, 276.5, 276.69, 276.34] [276.66, 276.5, 276.34]
2015-10-23 02:05:00
['U', 'U', 'S', 'S', 'U', 'S']
[275.9, 275.8, 275.5, 275.61, 275.52] [

2015-10-26 11:45:00
['U', 'U', 'U', 'U', 'U', 'D']
[287, 286.89, 286.98, 286.21, 285.52] [287, 285.51, 285.78, 285.49, 285.42]
2015-10-26 12:00:00
['U', 'S', 'D', 'U', 'U', 'U']
[287.41, 286.94, 287.68, 287, 286.89] [287.07, 287, 285.51]
2015-10-26 13:00:00
['U', 'D', 'S', 'D', 'U', 'U']
[287.83, 287.68, 288, 288.14, 288.08, 287.73] [287.6, 288.08, 287.4]
2015-10-26 14:15:00
['U', 'U', 'U', 'D', 'U', 'U']
[283.77, 283.38, 283.31] [283.36, 282.71, 282.63]
2015-10-26 14:25:00
['U', 'D', 'U', 'U', 'U', 'D']
[284.04, 285, 283.77, 283.38] [283.97, 283.36, 282.71]
2015-10-26 19:10:00
['U', 'D', 'U', 'U', 'S', 'D']
[284.21, 284.49, 284.87, 284.25] [283.96, 284.49, 283.72]
2015-10-26 20:30:00
['U', 'U', 'U', 'U', 'D', 'S']
[285.21, 284.61, 284.44, 283.82] [284.77, 284.61, 284.25, 283.45]
2015-10-26 20:50:00
['U', 'S', 'S', 'D', 'U', 'U']
[285.25, 284.73, 284.73, 285.51, 285.21, 284.61] [285.25, 284.77, 284.61]
2015-10-26 23:20:00
['U', 'D', 'S', 'U', 'S', 'U']
[288.13, 287.75, 288.13, 288.13, 

2015-10-29 07:00:00
['U', 'U', 'S', 'U', 'D', 'D']
[305.1, 304.99, 303.3, 303.3] [305.1, 304.5, 303.3]
2015-10-29 07:25:00
['U', 'S', 'U', 'U', 'D', 'U']
[306, 305.92, 305.92, 304.99] [305.93, 304.5, 303.2]
2015-10-29 07:40:00
['U', 'U', 'D', 'U', 'S', 'U']
[306.5, 306, 305.99, 306] [306.49, 306, 305.93]
2015-10-29 08:05:00
['U', 'D', 'U', 'U', 'D', 'U']
[306.68, 306.06, 306.69, 306.23] [306.36, 306.25, 305.62]
2015-10-29 08:15:00
['U', 'D', 'U', 'D', 'U', 'U']
[306.58, 306.58, 306.68, 306.06, 306.69] [306.58, 306.36, 306.25]
2015-10-29 08:45:00
['U', 'U', 'U', 'S', 'D', 'D']
[306.99, 306.95, 306.68, 305.97] [306.79, 306.72, 306.68]
2015-10-29 08:55:00
['U', 'D', 'U', 'U', 'U', 'S']
[307.23, 307.24, 306.99, 306.95] [306.75, 306.79, 306.72]
2015-10-29 09:55:00
['U', 'U', 'U', 'U', 'U', 'U']
[306.15, 305.97, 305, 304.39, 304.49, 304.3] [305.57, 305.2, 304.96, 304.39, 303.82, 303.34]
2015-10-29 10:05:00
['U', 'D', 'U', 'U', 'U', 'U']
[306.01, 306.08, 306.15, 305.97] [305.84, 305.57, 305.2

...

2016-06-29 02:55:00
['U', 'U', 'D', 'U', 'S', 'D']
[649.82, 649.82, 649.08, 649.98] [649.62, 649.03, 648.02]
2016-06-29 04:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[647.26, 646.22, 645.5] [646.04, 645.63, 644.61]
2016-06-29 05:20:00
['U', 'S', 'D', 'D', 'U', 'U']
[646.47, 640.58, 643.97, 644.46, 644.46, 644.28] [642.74, 644.21, 641.83]
2016-06-29 05:35:00
['U', 'U', 'D', 'U', 'S', 'D']
[647.5, 646.68, 646.01, 646.47] [646.81, 645.02, 642.74]
2016-06-29 07:10:00
['U', 'U', 'U', 'D', 'U', 'U']
[655.5, 649.7, 648.29] [650.4, 648.24, 647]
2016-06-29 08:05:00
['U', 'U', 'U', 'D', 'S', 'U']
[649.19, 648.78, 646.46] [649.13, 648.77, 646.46]
2016-06-29 09:25:00
['U', 'U', 'U', 'U', 'D', 'D']
[644.26, 642.82, 643.4, 641.99] [643, 642.67, 641.8, 639.89]
2016-06-29 10:20:00
['U', 'D', 'U', 'U', 'D', 'S']
[642.32, 642.57, 641.4, 641.07] [641.45, 640.66, 638.9]
2016-06-29 11:25:00
['U', 'U', 'U', 'D', 'D', 'D']
[628, 628.17, 623.64] [626.89, 624.62, 617.68]
2016-06-29 11:40:00
['U', 'D', 'D', 'U', 'U',

2016-07-02 02:20:00
['U', 'S', 'S', 'U', 'U', 'U']
[677.45, 676.67, 676.67, 676.67, 676.25] [676.37, 676.09, 676.01]
2016-07-02 02:45:00
['U', 'U', 'U', 'U', 'D', 'U']
[683.29, 683.11, 681.03, 679] [682.71, 680.1, 679.15, 677.5]
2016-07-02 04:10:00
['U', 'U', 'U', 'U', 'D', 'U']
[676.99, 675.94, 674.54, 673.88] [674.9, 675, 674.45, 672.88]
2016-07-02 05:00:00
['U', 'U', 'U', 'S', 'S', 'U']
[677.38, 677.15, 676.46, 676.28] [677.38, 677.08, 676.46]
2016-07-02 06:05:00
['U', 'U', 'S', 'U', 'D', 'D']
[676.95, 674.81, 674.14, 675.67] [676.81, 674.81, 674.14]
2016-07-02 06:45:00
['U', 'U', 'U', 'U', 'D', 'S']
[681.23, 679, 678.48, 677.69] [679.96, 678.45, 677.5, 676.88]
2016-07-02 07:55:00
['U', 'D', 'U', 'S', 'S', 'U']
[681.99, 679.83, 679.57, 678.41, 678.41, 678.87] [678.96, 679.32, 678.41]
2016-07-02 08:45:00
['U', 'U', 'U', 'U', 'D', 'S']
[679.23, 679.74, 678.35, 678.17] [679.05, 678.62, 678.01, 677]
2016-07-02 10:25:00
['U', 'U', 'D', 'U', 'U', 'S']
[681, 679.6, 679.28, 679.39] [679.78,

2016-07-04 16:25:00
['U', 'D', 'D', 'U', 'U', 'D']
[673.82, 673.94, 674.15, 674.22, 674] [673.81, 673, 672.03]
2016-07-04 17:05:00
['U', 'D', 'U', 'S', 'U', 'D']
[674.03, 672.01, 673.11, 671.83, 672] [672, 671.9, 671]
2016-07-04 17:40:00
['U', 'U', 'D', 'D', 'U', 'D']
[672.79, 671.99, 671.54, 670.9, 671.84] [672, 671.99, 671.84]
2016-07-04 19:35:00
['U', 'U', 'U', 'D', 'U', 'S']
[672.94, 671.63, 670.96] [672.32, 670.99, 669.83]
2016-07-04 22:25:00
['U', 'U', 'U', 'S', 'S', 'U']
[679.98, 678.83, 671.52, 669.53] [675.78, 674.1, 671.52]
2016-07-04 22:40:00
['U', 'U', 'D', 'U', 'U', 'U']
[680.34, 678.38, 678.48, 679.98] [677.65, 678.15, 675.78]
2016-07-05 00:50:00
['U', 'S', 'U', 'D', 'S', 'U']
[679.6, 678.1, 678.1, 676.88, 677.48, 677.48] [678.41, 678.08, 675.93]
2016-07-05 01:40:00
['U', 'U', 'D', 'U', 'D', 'U']
[673.12, 669.63, 668.98, 670.63] [670.76, 669.63, 668.9]
2016-07-05 03:35:00
['U', 'D', 'D', 'S', 'U', 'U']
[670.5, 668.31, 668.38, 670, 670, 669.99] [669, 670, 668.66]
2016-07-0

2016-07-08 01:20:00
['U', 'U', 'D', 'U', 'D', 'D']
[636.51, 636.23, 636.27, 636.71] [636.22, 636.22, 633.7]
2016-07-08 02:25:00
['U', 'U', 'D', 'U', 'U', 'D']
[624.92, 622.32, 622.76, 621.69] [621.49, 616.3, 612.78]
2016-07-08 02:35:00
['U', 'D', 'U', 'U', 'D', 'U']
[620.8, 623.56, 624.92, 622.32] [618.49, 621.49, 616.3]
2016-07-08 03:15:00
['U', 'U', 'D', 'D', 'U', 'D']
[619, 618.21, 618.1, 617.8, 617.92] [617.33, 617.05, 617.01]
2016-07-08 03:35:00
['U', 'U', 'D', 'D', 'U', 'U']
[620.61, 618.86, 618.82, 618.98, 619] [618.87, 617.62, 617.33]
2016-07-08 03:45:00
['U', 'D', 'U', 'U', 'D', 'D']
[620, 620.71, 620.61, 618.86] [619.51, 618.87, 617.62]
2016-07-08 04:15:00
['U', 'U', 'U', 'U', 'D', 'D']
[618.55, 618.4, 618.74, 617.85] [618.03, 616.58, 617.63, 616.51]
2016-07-08 07:05:00
['U', 'U', 'U', 'U', 'U', 'D']
[627.21, 626.28, 619.81, 621, 618.25] [624.98, 619.59, 616, 618, 609]
2016-07-08 07:40:00
['U', 'U', 'U', 'D', 'U', 'U']
[636.49, 632.98, 632.91] [632.39, 629.63, 627.66]
2016-07

2016-07-11 21:30:00
['U', 'U', 'U', 'U', 'D', 'U']
[644.57, 644.61, 644.21, 642.67] [643.5, 643.45, 642.91, 641.97]
2016-07-11 22:10:00
['U', 'U', 'U', 'D', 'D', 'S']
[645.29, 644.48, 643.88] [644.5, 643.98, 643.51]
2016-07-11 22:45:00
['U', 'U', 'U', 'U', 'S', 'U']
[647.58, 645.61, 645.57, 645.61, 644.22] [645.5, 645.6, 645, 644.66]
2016-07-11 23:45:00
['U', 'D', 'U', 'U', 'D', 'D']
[647.49, 647.49, 647.56, 647.42] [647.38, 647.5, 646.4]
2016-07-12 03:45:00
['U', 'D', 'S', 'U', 'U', 'U']
[649.32, 648.01, 649.3, 649.3, 648.82] [648.3, 648.66, 648.23]
2016-07-12 05:25:00
['U', 'U', 'U', 'D', 'D', 'U']
[648.38, 647.97, 647.7] [648.16, 647.69, 647.42]
2016-07-12 05:50:00
['U', 'S', 'U', 'S', 'D', 'U']
[649.49, 648.3, 648.3, 647.52, 647.6, 648.38] [648.42, 648.3, 648.16]
2016-07-12 17:55:00
['U', 'D', 'D', 'U', 'D', 'U']
[649.95, 649.12, 650, 650.4, 647.88, 647.62] [649.1, 649.5, 647.59]
2016-07-12 18:25:00
['U', 'D', 'U', 'U', 'S', 'D']
[651.63, 649.04, 649.5, 649.02] [649.5, 649.5, 649.0